# Plotting topography and other uneven distributed arrays

In [ ]:
# Imports

import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import roppy

In [ ]:
# Data file
roms_file = "data/ocean_avg_example.nc"

In [ ]:
# Read the bathymetry
with Dataset(roms_file) as ncid:
    H = ncid.variables["h"][:, :]
    M = ncid.variables["mask_rho"][:, :]

The main problem is that details in the shallow North Sea are not visible with a straight coloured contour map.

In [ ]:
# First stab at plotting the bathymetry
plt.contourf(H)
plt.colorbar()

The contour functions can take a level argument, but that does not help much with the colour filling.
The colour map is reversed to make the deep sea dark blue. `axis(image)` gives the plot the ration between 
with and height.

In [ ]:
L = [25, 50, 100, 250, 500, 1000, 2500]
plt.contourf(H, levels=L, cmap="viridis_r", extend="both")
plt.colorbar()

plt.axis("image");

An alternative for bathymetry is to take the logarithm. This shows the details in the shallow North Sea.
The colour map is messed up and is not easily fixed. Some thin black isolines are added for clarity. 
The land is still not masked out.

In [ ]:
L = [25, 50, 100, 250, 500, 1000, 2500]
plt.contourf(np.log10(H), levels=np.log10(L), cmap="viridis_r", extend="both")
plt.colorbar()
plt.contour(H, levels=L, colors="black", linewidths=0.5)
plt.axis("image");


An alternative to the logarithm is to take explicit control over the colour map and the normalization and spread out the desired levels evenly along the colour axis. This is done by the function `levelmap` in roppy.
This is a more general applicable solution for other variables than taking the logarithm.
Land can be masked out by setting the data values to NaN. A poor man's coast line is obtained by contouring the land mask at level 0.5 between land and sea.


In [ ]:
H2 = np.where(M > 0, H, np.nan)  # Mask out land
extend = "both"
cmap, norm = roppy.levelmap(L, extend=extend, reverse=True)
cmap.set_bad("grey")
plt.contourf(H2, levels=L, cmap=cmap, norm=norm, extend=extend)
plt.colorbar()
plt.contour(H, levels=L, colors="black", linewidths=0.5)
plt.contour(M, levels=[0.5], colors="black")  # Coast line
plt.axis("image");

Roppy has a function `landmask` for adding the grid's land mask.

In [ ]:
extend = "both"
cmap, norm = roppy.levelmap(L, extend=extend, reverse=True)
plt.contourf(H, levels=L, cmap=cmap, norm=norm, extend=extend)
plt.colorbar()
plt.contour(H, levels=L, colors="black", linewidths=0.5)

# Roppy has a function for making a land mask
roppy.landmask(M, "grey")

plt.axis("image")


The same `cmap`and `norm` can be used for `pcolormesh` as well.

In [ ]:
H2 = np.where(M > 0, H, np.nan)  # mask out land

cmap, norm = roppy.levelmap(L, extend=extend, reverse=True)
cmap.set_bad("grey")  # Set a colour for undefined (land) cells.

# We need the grid cell boundaries
jmax, imax = M.shape
Xb = np.arange(-0.5, imax)
Yb = np.arange(-0.5, jmax)
plt.pcolormesh(Xb, Yb, H2, cmap=cmap, norm=norm)
plt.colorbar(extend="both")
# Add some contour lines
plt.contour(H, levels=[100, 1000], colors="black", linewidths=0.5)

plt.axis("image");